<a id="top"></a>

The purpose of this notebook is to create a dataset containing LINEID, ROUTEID, PROGRNUMBER & STOPPOINTID and to gain a better understanding of this data.

***

# Import Packages

In [1]:
import pandas as pd
import sqlite3

***

<a id="contents"></a>
# Contents

- [1. Connect to Database](#connect_to_db)
- [2. Query Database](#query_db)

***

<a id="connect_to_db"></a>
# 1. Connect to Database
[Back to contents](#contents)

In [2]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

In [3]:
# create connection to database
db = '/home/faye/Data-Analytics-CityRoute/dublinbus.db'
conn = create_connection(db)

***

<a id="query_db"></a>
# 2. Query Database
[Back to contents](#contents)

In [4]:
# initialise query
query = """
SELECT DISTINCT T.LINEID, T.ROUTEID, L.PROGRNUMBER, L.STOPPOINTID
FROM leavetimes L, trips T
WHERE L.TRIPID = T.TRIPID
"""

In [5]:
df_query = pd.read_sql(query, conn)

***

[Back to top](#top)